# Testing CNN Models

In this notebook, I will be training and tracking the performance of different image recognition / object detection / object segmentation etc starting from the basics. 

Some of the models that I use are my implementations of popular CNN models such as:

- MobileNetV1.py

In [1]:
# Import proper libraries
import os
import sys
import time

import torch
import torch.nn as nn
import torchsummary as summary

# Detect devices (as well as mps for arm macos)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# Custom libraries
import importlib.util

def spec_loader(module_path, module_name):
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

## Simple MLP
Let's have the most simple ANN. A MLP neural network and see its performance on handwritten image set MNIST.

In [3]:
# Load Data
mnist = spec_loader('./utils/image_data_loaders.py', 'MNISTDataLoader')
mnist_loader = mnist.MNISTDataLoader(batch_size = 64)

train_loader, test_loader = mnist_loader.get_data_loaders()
print("Length of Train", len(train_loader), "Shape of Train", train_loader.dataset.data.shape)
print("Length of Test", len(test_loader), "Shape of Test", test_loader.dataset.data.shape)

# Print first 10 labels
print("First 10 labels in the training set")
# for i in range(10):
#     print(train_loader.dataset.targets[i])

# Reshape the data to fit into the model
train_loader = [(x.reshape(-1, 28*28), y) for x, y in train_loader]
test_loader = [(x.reshape(-1, 28*28), y) for x, y in test_loader]

Length of Train 938 Shape of Train torch.Size([60000, 28, 28])
Length of Test 157 Shape of Test torch.Size([10000, 28, 28])
First 10 labels in the training set


In [4]:
# Model
module_path = './03_Recreating_CNN_Models/SimpleMLP.py'
simple_mlp = spec_loader(module_path, 'SimpleMLP')
model = simple_mlp.SimpleMLP(input_size=784, hidden_size=500, num_classes=10)

# Import trainer helper function
trainer = spec_loader('./utils/pytorch_train_template.py', 'Training')
trainer = trainer.Training(
    model = model,
    train_loader = train_loader,
    test_loader = test_loader,
    criterion = nn.CrossEntropyLoss(),
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001),
    num_epochs = 5,
    new_train_dir = "./out/SimpleMLP/"
)

trainer.start_training()

Epoch 1/5


100%|██████████| 157/157 [00:00<00:00, 4469.08it/s]

Train Accuracy: 60.059701492537314, Train Loss: 0.20305666417630117
Test Accuracy: 61.57324840764331, Test Loss: 0.1057655869146726
Epoch 2/5



100%|██████████| 157/157 [00:00<00:00, 4436.38it/s]

Train Accuracy: 62.346481876332625, Train Loss: 0.08277704467280095
Test Accuracy: 62.05732484076433, Test Loss: 0.08196814471535432
Epoch 3/5


...

Train Accuracy: 62.93923240938166, Train Loss: 0.05233331520863965
Test Accuracy: 62.35668789808917, Test Loss: 0.06860047741385272
Epoch 4/5



100%|██████████| 157/157 [00:00<00:00, 4452.55it/s]

Train Accuracy: 63.26759061833689, Train Loss: 0.035385933716712926
Test Accuracy: 62.254777070063696, Test Loss: 0.07416734800846049
Epoch 5/5



100%|██████████| 157/157 [00:00<00:00, 4475.34it/s]

Train Accuracy: 63.37420042643923, Train Loss: 0.028277178129828127
Test Accuracy: 62.10828025477707, Test Loss: 0.08663639272713312
Training finished
--------------------------------------------------


## SimpleCNN
Now let's compare this with the CNN models that are proposed in the paper "Esemble of CNNs for Hnadwritten Digit Recognition".

In [16]:
# Get the model
simple_cnn = spec_loader('./03_Recreating_CNN_Models/MnistK3.py', 'ModelM3')
simple_cnn = simple_cnn.ModelM3(in_channels=1, num_classes=10)

# Load the dataset
mnist_loader = mnist.MNISTDataLoader(batch_size = 64)
train_loader, test_loader = mnist_loader.get_data_loaders()

# Train the model
trainer = spec_loader('./utils/pytorch_train_template.py', 'Training')
trainer = trainer.Training(
    model = simple_cnn,
    train_loader = train_loader,
    test_loader = test_loader,
    criterion = nn.CrossEntropyLoss(),
    optimizer = torch.optim.Adam(simple_cnn.parameters(), lr=0.01),
    num_epochs = 5,
    new_train_dir = "./out/SimpleCNN/"
)
trainer.start_training()

Epoch 1/5


  0%|          | 0/938 [00:00<?, ?it/s]

100%|██████████| 157/157 [00:03<00:00, 39.29it/s]

Train Accuracy: 61.21535181236674, Train Loss: 0.18035050448395615
Test Accuracy: 62.560509554140125, Test Loss: 0.06425977997618258
Epoch 2/5



 73%|███████▎  | 682/938 [00:55<00:20, 12.49it/s]